<a href="https://colab.research.google.com/github/natalymogollon/Capstone---G11/blob/main/Capstone_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **USIL_Capstone_202301_G11**

#### Integrantes:
- Canales Del Rio, Ana Belen
- Mogollon Bermudez, Nataly Karla

Titulo: Un estudio comparativo de las técnicas computacionales para la detección de fake reviews en los productos de Amazon

Descripción: Código en Python para la ejecución de dos métodos computacionales para la detección de fake reviews y su comparación para analizar, en el determinado número de datos, cuál tuvo un porcentaje más alto de precisión en la detección de las reseñas falsas.

El dataset seleccionado cuenta con aproximadamente más de 568 000 registros, por lo que seleccionamos una media de 25 000 reviews para entrenar en los modelos empleados y tener un resultado más rápido.

Dataset utilizado: "Reviews_25k.csv"

Fuente del dataset: https://www.kaggle.com/code/ak1982/amazon-review-sentiment-analysis-prediction/notebook

Fuentes de los métodos mencionados:

SF-BiLSTM: https://inass.org/wp-content/uploads/2022/06/2022103141-2.pdf

NBSVM: https://www.sciencedirect.com/science/article/pii/S0969698921003374

## **Ejecución Modelo SF-BiLSTM**



In [ ]:
# Importación de Librerías necesarias para el modelo
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional, SpatialDropout1D
from sklearn.metrics import precision_score, recall_score


In [ ]:
!pip install pandas
!pip install numpy
!pip install nltk
!pip install sklearn
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post5-py3-none-any.whl size=2950 sha256=4e2cf08f4f172f6ad28f20b988fdae5c711f5ab41a8f7b715ac87744d43d5b0a
  Stored in directory: /root/.cache/pip/wheels/38/1f/8d/4f812c590e074c1e928f5cec67bf5053b71f38e2648739403a
Successfully built sklearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Codigo para la carga de archivos dentro de google.colab
from google.colab import files
uploaded = files.upload()

Saving Reviews_25k.csv to Reviews_25k.csv


In [ ]:
#dataset:
data = pd.read_csv("Reviews_25k.csv", sep=';')

num_reviews = len(data)
print("Total de reviews:", num_reviews)

Total de reviews: 25000


In [ ]:
data

In [ ]:
# Descargar los recursos necesarios para NLTK
nltk.download('punkt')

In [ ]:
# Dividir los datos en entrenamiento y prueba
train_data, test_data, train_scores, test_scores = train_test_split(
    data['Text'], data['Score'], test_size=0.2, random_state=42)

# Preprocesamiento de los textos
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data)

# Convertir los textos a secuencias numéricas
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data)
train_sequences = tokenizer.texts_to_sequences(train_data)
test_sequences = tokenizer.texts_to_sequences(test_data)

# Ajustar la longitud de las secuencias
max_sequence_length = 100
train_padded = pad_sequences(train_sequences, maxlen=max_sequence_length)
test_padded = pad_sequences(test_sequences, maxlen=max_sequence_length)

# Crear el modelo SF-BiLSTM
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length))
model.add(SpatialDropout1D(0.5))
model.add(Bidirectional(LSTM(64, dropout=0.5, recurrent_dropout=0.5)))
model.add(Dense(6, activation='softmax'))


# Compilar el modelo
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(train_padded, train_scores, epochs=10, batch_size=32, validation_data=(test_padded, test_scores))

# Evaluar el modelo
predictions_bilstm = np.argmax(model.predict(test_padded), axis=-1)
accuracy = model.evaluate(test_padded, test_scores)[1]
precision_bilstm = precision_score(test_scores, predictions_bilstm, average='weighted')
recall_bilstm = recall_score(test_scores, predictions_bilstm, average='weighted')

# Obtener la cantidad de fake reviews detectadas
fake_reviews_bilstm = np.count_nonzero(predictions_bilstm)

print('Accuracy:', accuracy)
print('Precision:', precision_bilstm)
print('Recall:', recall_bilstm)
print('Fake Reviews Detected:', fake_reviews_bilstm)



## **Ejecución Modelo NBSVM**

In [ ]:
import pandas as pd
import time

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

In [ ]:
!pip install pandas
!pip install sklearn

In [ ]:
# Codigo para la carga de archivos dentro de google.colab
from google.colab import files
uploaded = files.upload()

In [ ]:
#dataset:
data = pd.read_csv("Reviews_25k.csv", sep=';')

num_reviews = len(data)
print("Total de reviews:", num_reviews)

In [ ]:
# Dividir el dataset en conjunto de entrenamiento y prueba
X = data['Text']
y = data['Score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el pipeline para el modelo NBSVM
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('classifier', SVC(kernel='linear', probability=True))
])

# Entrenar el modelo y medir el tiempo
print("Entrenando el modelo...")
start_time = time.time()
pipeline.fit(X_train, y_train)
end_time = time.time()
training_time = end_time - start_time
print("Entrenamiento completado. Tiempo de entrenamiento:", training_time, "segundos")

# Realizar predicciones en el conjunto de prueba
y_pred = pipeline.predict(X_test)

# Calcular las métricas de precisión y recall
precision_nbsvm = precision_score(y_test, y_pred, average='micro')
recall_nbsvm = recall_score(y_test, y_pred, average='micro')

# Imprimir los resultados
print("Resultados:")
print("----------")
print("Precisión:", precision_nbsvm)
print("Recall:", recall_nbsvm)

# Contar la cantidad de fake reviews
fake_reviews_nbsvm = sum(y_pred < 3)
print("Cantidad de fake reviews detectadas:", fake_reviews_nbsvm)

## **Comparación de Resultados**

In [ ]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score


In [ ]:
# Resultados del modelo NBSVM
nbsvm_precision = precision_nbsvm
nbsvm_recall = recall_nbsvm
nbsvm_fake_reviews = fake_reviews_nbsvm

# Resultados del modelo SF-BiLSTM
sfbilstm_precision = precision_bilstm
sfbilstm_recall = recall_bilstm
bilstm_fake_reviews = fake_reviews_bilstm

In [ ]:

# Crear un DataFrame para almacenar los resultados
data_compare = {
    'Modelo': ['NBSVM', 'SF-BiLSTM'],
    'Precisión': [nbsvm_precision, sfbilstm_precision],
    'Recall': [nbsvm_recall, sfbilstm_recall],
    'Fake Reviews Detectadas': [nbsvm_fake_reviews, bilstm_fake_reviews]
}
df = pd.DataFrame(data_compare)

# Comparar los resultados
print(df)

# Conclusiones
if nbsvm_precision > sfbilstm_precision:
    print("El modelo NBSVM tiene una precisión mayor.")
elif nbsvm_precision < sfbilstm_precision:
    print("El modelo SF-BiLSTM tiene una precisión mayor.")
else:
    print("Ambos modelos tienen la misma precisión.")

if nbsvm_recall > sfbilstm_recall:
    print("El modelo NBSVM tiene un recall mayor.")
elif nbsvm_recall < sfbilstm_recall:
    print("El modelo SF-BiLSTM tiene un recall mayor.")
else:
    print("Ambos modelos tienen el mismo recall.")

# Determinar el mejor modelo basado en la cantidad de fake reviews detectadas
if bilstm_fake_reviews > nbsvm_fake_reviews:
    print("El modelo SF-BiLSTM ha detectado más fake reviews.")
elif bilstm_fake_reviews < nbsvm_fake_reviews:
    print("El modelo NBSVM ha detectado más fake reviews.")
else:
    print("Ambos modelos han detectado la misma cantidad de fake reviews.")

